In [1]:
#pip install streamlit

In [11]:
import pdfplumber
import pytesseract
from pdf2image import convert_from_path
import re
import cv2
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET
import os
import glob

def extract_from_page(page_number, page):
    texte = page.extract_text()
    if texte == "":
        # Convertir l'image en niveaux de gris
        gray = cv2.cvtColor(np.array(images[page_number]), cv2.COLOR_RGB2GRAY)
        
        # Apply OCR to get the OSD (Orientation and Script Detection) information
        data = pytesseract.image_to_osd(gray)
        # Extract the rotation angle from the OSD data
        lines = data.splitlines()
        rotation_line = next((line for line in lines if line.startswith('Rotate:')), None)
        rotation_angle = int(rotation_line.split(':')[1])
        # Rotate the image to correct the text rotation
        if rotation_angle != 0:
            if rotation_angle == 90:
                gray = cv2.rotate(gray, cv2.ROTATE_90_CLOCKWISE)
            elif rotation_angle == 180:
                gray = cv2.rotate(gray, cv2.ROTATE_180)
            elif rotation_angle == 270:
                gray = cv2.rotate(gray, cv2.ROTATE_90_COUNTERCLOCKWISE)
                
        # Appliquer des techniques de prétraitement d'image pour nettoyer les traces de stylos
        # Par exemple, appliquer un filtre de flou pour réduire le bruit
        blurred = cv2.GaussianBlur(gray, (5, 5), 0)
        # Appliquer une segmentation en seuillage adaptatif pour isoler le texte
        _, threshold = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        
        # Effectuer OCR sur l'image traitée
        texte = pytesseract.image_to_string(threshold, lang='eng', config='--psm 4')
        
    lines = texte.split("\n")
    aev_words = []
    voyage_words = []
    vessel_words = []
    flag_words = []
    loading_words = []
    discharge_words = []
    first_line_below_bl_total = []
    second_line_below_bl_total = []
    sh_words = []
    cn_words = []
    no_words = []
    regex_matched_text = []
    #next_lines = []
    
    #regex_pattern = r"(\d{2}X\s*\d{2}\'*\s*[A-Z]+\s*[A-Z]{3})(?:.*\n){3}"
    
    for i, line in enumerate(lines):
        patern=r'\b(\d+\s*X\s*\d+\'*[A-Za-z]+\s*\w+\s*\w+\s*\w+\s*\w+)\b'
        #regex_match = re.findall(r"(\d{2}X\s*\d{2}\'*\s*[A-Z]+\s*[A-Z]{3})", line)
        regex_match = re.findall(patern, line)
        regex_matched_text.extend(regex_match)
        if regex_match:
            regex_matched_text.extend(regex_match)
            
            # Concaténer les trois lignes suivantes dans une seule chaîne de caractères
            next_lines = lines[i+1:i+4]
            concatenated_lines = ' '.join(next_lines)
            
            regex_matched_text.append(concatenated_lines)
        #if regex_match:
            #regex_matched_tex = regex_match.group(1)
            #regex_matched_text.append(" ".join(regex_matched_tex.groups()))
        
        words = re.findall(r'^[A-Z]{1,4}\d{6,}', line)
        aev_words.extend(words)
        
        voyage_match = re.search(r'VOYAGE:\s*(\w+)', line)
        if voyage_match:
            voyage_word = voyage_match.group(1)
            voyage_words.append(voyage_word)
        
        vessel_match = re.search(r'VESSEL:\s*(\w+)', line)
        if vessel_match:
            vessel_word = vessel_match.group(1)
            vessel_words.append(vessel_word)
        
        flag_match = re.search(r'FLAG:\s*(\w+)', line)
        if flag_match:
            flag_word = flag_match.group(1)
            flag_words.append(flag_word)
        
        loading_match = re.search(r'Actual Port of Loading\s*(\w+)', line)
        if loading_match:
            loading_word = loading_match.group(1)
            loading_words.append(loading_word)
        
        discharge_match = re.search(r'Actual Port of Discharge\s*(\w+)', line)
        if discharge_match:
            discharge_word = discharge_match.group(1)
            discharge_words.append(discharge_word)
        
        sh_match = re.search(r'SH:\s*(\w+)\s*(\w+)\s*(\w+)\s*(\w+)', line)
        if sh_match:
            sh_words.append(" ".join(sh_match.groups()))
        
        cn_match = re.search(r'CN:\s*(\w+)\s*(\w+)\s*(\w+)\s*(\w+)', line)
        if cn_match:
            cn_words.append(" ".join(cn_match.groups()))
        
        no_match = re.search(r'^N:\s*(\w+)\s*(\w+)\s*(\w+)\s*(\w+)', line)
        if no_match:
            no_words.append(" ".join(no_match.groups()))
        
        if "ITEMS:" in line:
            first_line_below_bl_total.append(line)
        
        if "SPLITS:" in line:
            second_line_below_bl_total.append(line)
        
        ## Récupérer le format "12X40 ST STC" et les trois lignes suivantes
       # regex_match = re.search(r"(\d{2}X\s*\d{2}\'*\s*[A-Z]+\s*[A-Z]{3})(?:.*\n){3}", line)
        #if regex_match:
            #regex_matched_tex = regex_match.group(1)
            #regex_matched_text.append(" ".join(regex_matched_tex.groups()))
            # Récupérer les trois lignes suivantes
            #next_lines = lines[i+1:i+4]
    
    if not aev_words:
        aev_words.append("N/A")
    if not voyage_words:
        voyage_words.append("N/A")
    if not vessel_words:
        vessel_words.append("N/A")
    if not flag_words:
        flag_words.append("N/A")
    if not loading_words:
        loading_words.append("N/A")
    if not discharge_words:
        discharge_words.append("N/A")
    if not first_line_below_bl_total:
        first_line_below_bl_total.append("N/A")
    if not second_line_below_bl_total:
        second_line_below_bl_total.append("N/A")
    if not sh_words:
        sh_words.append("N/A")
    if not cn_words:
        cn_words.append("N/A")
    if not no_words:
        no_words.append("N/A")
    if not regex_matched_text:
        regex_matched_text.append("N/A")
    #if not next_lines:
        # next_lines.append("N/A")
    
    
    return (
        aev_words, voyage_words, vessel_words, flag_words, loading_words,
        discharge_words, first_line_below_bl_total, second_line_below_bl_total,
        sh_words, cn_words, no_words, regex_matched_text
    )


# Chemin vers le dossier contenant les fichiers PDF
dossier_pdf = 'C:\\Users\\ceiba\\Desktop\\PDF_EXTRACT\\PDF'
# Chemin vers le dossier contenant les fichiers XML
dossier_xml ='C:\\Users\\ceiba\\Desktop\\PDF_EXTRACT\\PDF'
# Chemin vers le dossier contenant les fichiers CSV
dossier_csv = 'C:\\Users\\ceiba\\Desktop\\PDF_EXTRACT\\PDF'

# Rechercher tous les fichiers PDF dans le dossier
chemins_pdf = glob.glob(os.path.join(dossier_pdf, '*.pdf'))

# Parcourir chaque fichier PDF trouvé
for chemin_pdf in chemins_pdf:
    # Convertir les pages PDF en images en utilisant pdf2image
    images = convert_from_path(chemin_pdf, dpi=600)
    # Ouvrir le fichier PDF avec PDFplumber
    with pdfplumber.open(chemin_pdf) as pdf:
        aev_words_list = []
        voyage_words_list = []
        vessel_words_list = []
        flag_words_list = []
        loading_words_list = []
        discharge_words_list = []
        first_line_below_bl_total_list = []
        second_line_below_bl_total_list = []
        sh_words_list = []
        cn_words_list = []
        no_words_list = []
        regex_matched_text_list = []
        #next_lines_list = []
        
        
        for page_number, page in enumerate(pdf.pages):
            page_aev_words, page_voyage_words, page_vessel_words, page_flag_words, page_loading_words, page_discharge_words, page_first_line_below_bl_total, page_second_line_below_bl_total, page_sh_words, page_cn_words, page_no_words, page_regex_matched_text = extract_from_page(page_number, page)
            aev_words_list.extend(page_aev_words)
            voyage_words_list.extend(page_voyage_words)
            vessel_words_list.extend(page_vessel_words)
            flag_words_list.extend(page_flag_words)
            loading_words_list.extend(page_loading_words)
            discharge_words_list.extend(page_discharge_words)
            first_line_below_bl_total_list.extend(page_first_line_below_bl_total)
            second_line_below_bl_total_list.extend(page_second_line_below_bl_total)
            sh_words_list.extend(page_sh_words)
            cn_words_list.extend(page_cn_words)
            no_words_list.extend(page_no_words)
            regex_matched_text_list.extend(page_regex_matched_text)
            #next_lines_list.extend(page_next_lines)
            
            # Vérifier la longueur des listes
            list_lengths = [len(aev_words_list), len(voyage_words_list), len(vessel_words_list), len(flag_words_list), len(loading_words_list), len(discharge_words_list), len(first_line_below_bl_total_list), len(second_line_below_bl_total_list), len(sh_words_list), len(cn_words_list), len(no_words_list), len(regex_matched_text_list)]
            max_length = max(list_lengths)
            
            # Remplir les listes manquantes avec "N/A" pour atteindre la même longueur
            aev_words_list += ["N/A"] * (max_length - len(aev_words_list))
            voyage_words_list += ["N/A"] * (max_length - len(voyage_words_list))
            vessel_words_list += ["N/A"] * (max_length - len(vessel_words_list))
            flag_words_list += ["N/A"] * (max_length - len(flag_words_list))
            loading_words_list += ["N/A"] * (max_length - len(loading_words_list))
            discharge_words_list += ["N/A"] * (max_length - len(discharge_words_list))
            first_line_below_bl_total_list += ["N/A"] * (max_length - len(first_line_below_bl_total_list))
            second_line_below_bl_total_list += ["N/A"] * (max_length - len(second_line_below_bl_total_list))
            sh_words_list += ["N/A"] * (max_length - len(sh_words_list))
            cn_words_list += ["N/A"] * (max_length - len(cn_words_list))
            no_words_list += ["N/A"] * (max_length - len(no_words_list))
            regex_matched_text_list += ["N/A"] * (max_length - len(regex_matched_text_list))
            #next_lines_list += ["N/A"] * (max_length - len(next_lines_list))
            
        # Créer un DataFrame à partir des listes de mots AEV0188950, Voyage, Vessel, Flag et Actual Port of Loading
    df = pd.DataFrame({"Voyage": voyage_words_list, "BL": aev_words_list, "Vessel": vessel_words_list, "Flag": flag_words_list, "Actual_Port_of_Loading": loading_words_list, "Actual_Port_of_discharge": discharge_words_list, "Number_line1": first_line_below_bl_total_list, "Number_line2": second_line_below_bl_total_list, "Shipper": sh_words_list, "Consigner": cn_words_list, "Notifie": no_words_list, "Description" : regex_matched_text_list})
    # Diviser le champ "Nom Complet" en prénom, deuxième prénom, nom de famille et suffixe
    df[['ITEMS', 'Type_contener', 'PACKS', 'Nbre_colis', 'WGT', 'Poids_march','Unite_poids']] = df['Number_line1'].str.split(" ", 6, expand=True)
    df[['SPLITS', 'Nbre_contener', 'IN_PACKS', 'Nbre_INPACKS', 'VOL', 'Volum_march','Unite_volum']] = df['Number_line2'].str.split(" ", 6, expand=True)
    # Supprimer les champs "Prénom" et "Nom de Famille"
    champs_a_supprimer = ["ITEMS", "PACKS", "WGT", "Number_line1", "SPLITS", "IN_PACKS", "Nbre_INPACKS", "VOL", "Number_line2"]
    df = df.drop(columns=champs_a_supprimer)
    # Extraire le nom du fichier
    Id_fichier = os.path.basename(chemin_pdf)
    # Ajouter la colonne "Nom du Fichier" avec la valeur du nom du fichier
    df['Nom_du_Fichier'] = Id_fichier
############################################################################################
    last_bl_index = df['BL'].last_valid_index()
# Parcourir la colonne "BL" et remplacer chaque "N/A" par la dernière valeur non nulle
    for i, value in enumerate(df['BL']):
        if value == "N/A":
            df.at[i, 'BL'] = df.at[last_bl_index, 'BL']
        else:
            last_bl_index = i  # Mettre à jour l'index de la dernière valeur non nulle
############################################################################################
############################################################################################
   
    # chaque occurrence de "N/A" sera remplacée par la dernière valeur non nulle trouvée avant celle-ci.
    df = df.groupby("BL").agg({
        "Voyage": "first", 
        "Vessel": "first", 
        "Flag": "first",  
        "Actual_Port_of_Loading": "first", 
        "Actual_Port_of_discharge": "first", 
        "Shipper": "first",  
        "Consigner": "first", 
        "Notifie": "first", 
        "Description":lambda x: list(set(x)),  # Prendre les valeurs uniques de "Description" pour chaque groupe "BL"  
        "Type_contener": "last", 
        "Nbre_colis": "last", 
        "Poids_march": "last",  
        "Unite_poids": "last", 
        "Nbre_contener": "last",   
        "Volum_march": "last",
        "Unite_volum": "last",
        "Nom_du_Fichier": "last"
    }).reset_index()
    
    # Convertir les listes en chaînes de caractères dans le DataFrame
    #df['Description'] = ' '.join(map(str, df['Description']))
    df['Description'] = df['Description'].apply(lambda liste: ' '.join(map(str, liste)))
    
    ######################################################################################################
    root = ET.Element('manifest')
    
    # Conversion du dataframe en XML
    for _, row in df.iterrows():
        result = ET.SubElement(root, 'Result')
        
        ET.SubElement(result, 'Voyage').text = row['Voyage']
        ET.SubElement(result, 'BL').text = row['BL']
        ET.SubElement(result, 'Vessel').text = row['Vessel']
        ET.SubElement(result, 'Flag').text = row['Flag']
        ET.SubElement(result, 'Actual_Port_of_Loading').text = row['Actual_Port_of_Loading']
        ET.SubElement(result, 'Actual_Port_of_discharge').text = row['Actual_Port_of_discharge']
        ET.SubElement(result, 'Nbre_colis').text = row['Nbre_colis']
        ET.SubElement(result, 'Poids_march').text = row['Poids_march']
        ET.SubElement(result, 'Unite_poids').text = row['Unite_poids']
        ET.SubElement(result, 'Nbre_contener').text = row['Nbre_contener']
        ET.SubElement(result, 'Type_contener').text = row['Type_contener']
        ET.SubElement(result, 'Volum_march').text = row['Volum_march']
        ET.SubElement(result, 'Unite_volum').text = row['Unite_volum']
        ET.SubElement(result, 'Shipper').text = row['Shipper']
        ET.SubElement(result, 'Consigner').text = row['Consigner']
        ET.SubElement(result, 'Notifie').text = row['Notifie']
        ET.SubElement(result, 'Description').text = row['Description']
        
    # On peut ajouter d'autres éléments en fonction de la structure de votre dataframe
    
    # Enregistrer le XML dans un fichier avec un nom basé sur le fichier PDF
    nom_fichier_xml = os.path.splitext(os.path.basename(chemin_pdf))[0] + '.xml'
    chemin_fichier_xml = os.path.join(dossier_xml, nom_fichier_xml)
    # Création de l'arbre XML
    xml_tree = ET.ElementTree(root)
    # Enregistrement du XML dans un fichier
    xml_tree.write(chemin_fichier_xml, encoding='utf-8', xml_declaration=True)
    
    
    # Enregistrer le DataFrame en tant que fichier CSV avec un nom basé sur le fichier PDF
    nom_fichier_csv = os.path.splitext(os.path.basename(chemin_pdf))[0] + '.csv'
    chemin_fichier_csv = os.path.join(dossier_csv, nom_fichier_csv)
    df.to_csv(chemin_fichier_csv, index=False)

In [9]:
import pandas as pd

# Exemple de DataFrame avec des listes dans les cellules
donnees = {
    'colonne1': [1, 2, 3],
    'colonne2': ['a', 'b', 'c'],
    'colonne_liste': [[10, 20, 30], [40, 50], [60, 70, 80]]
}

mon_dataframe = pd.DataFrame(donnees)

# Convertir les listes en chaînes de caractères dans le DataFrame
mon_dataframe['colonne_liste'] = mon_dataframe['colonne_liste'].apply(lambda liste: ' '.join(map(str, liste)))

# Écrire le DataFrame dans un fichier CSV
mon_dataframe.to_csv('mon_fichier.csv', index=False)
